In [10]:
from typing import Tuple, List

In [12]:
# parse input
def make_criterias(A: List[int]) -> (List[int], List[int]):
    Q1 = []
    Q2 = []
    for i in A:
        assert(i < 100)
        Q1.append(i//10)
        Q2.append(i%10)
    return (Q1, Q2)

In [13]:
A = [79, 95, 4, 37, 92, 95, 12, 52, 70, 14]

In [29]:
Q1, Q2 = make_criterias(A)
print(" Alternatives ".center(len(Q1)*3, "#"))
print(f"Q1 = {Q1}")
print(f"Q2 = {Q2}")

######## Alternatives ########
Q1 = [7, 9, 0, 3, 9, 9, 1, 5, 7, 1]
Q2 = [9, 5, 4, 7, 2, 5, 2, 2, 0, 4]
